In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import pairwise

In [20]:
import pandas as pd

data = pd.read_csv('./improveddsv5.csv', encoding="ISO-8859-1")

In [39]:
BERT_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4" # @param {type: "string"} ["https://tfhub.dev/google/experts/bert/wiki_books/2", "https://tfhub.dev/google/experts/bert/wiki_books/mnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qqp/2", "https://tfhub.dev/google/experts/bert/wiki_books/squad2/2", "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2",  "https://tfhub.dev/google/experts/bert/pubmed/2", "https://tfhub.dev/google/experts/bert/pubmed/squad2/2"]
# Preprocessing must match the model, but all the above use the same.
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [40]:
#Rename the first column to "Question_ID"
data.rename(columns={data.columns[0]: "Question_ID"}, inplace=True)

In [46]:
#convert 'Score' to float
data['Score'] = data['Score'].astype(float)

In [47]:
preprocess = hub.load(PREPROCESS_MODEL)
bert = hub.load(BERT_MODEL)

In [48]:
# add new two columns to the dataframe Model_Answer_Embed and Answer_Embed
data['Model_Answer_Embed'] = data['Model_Answer'].apply(lambda x: bert(preprocess([x]))["pooled_output"])
data['Answer_Embed'] = data['Answer'].apply(lambda x: bert(preprocess([x]))["pooled_output"])


In [49]:
data['Cosine_Similarity'] = data.apply(lambda x: pairwise.cosine_similarity(x['Model_Answer_Embed'], x['Answer_Embed'])[0][0], axis=1)

In [2]:
data.isnull().sum()

Unnamed: 0      0
Question        0
Model_Answer    0
Answer          0
Score           0
Category        0
dtype: int64

In [19]:
data

,Unnamed: 0,Question,Model_Answer,Answer,Score,Category
0,1.0,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,This is used to use as a working system for th...,0.0,0
1,1.0,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is given as the final product to the client...,0.0,0
2,1.0,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,The prototype is use for teaching to the stude...,0.0,0
3,1.0,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,I don't have a clear idea on what is prototype,0.0,0
4,1.0,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,This is used to simulate the status of the pro...,0.0,0
...,...,...,...,...,...,...
496,13.4,What is doing in first normalization,In first normalization it removes all the dupl...,In first normalization remove all the duplicat...,3.0,1
497,13.4,What is doing in first normalization,In first normalization it removes all the dupl...,In first normalization remove all the duplicat...,0.0,0
498,13.4,What is doing in first normalization,In first normalization it removes all the dupl...,All the duplicated rows are removed in the fir...,0.0,0
499,13.4,What is doing in first normalization,In first normalization it removes all the dupl...,I don't have an idea about the question,0.0,0


In [52]:
ind = 1.1
new_index = 1
for index, row in data.iterrows():
    if(row['Question_ID'] == ind):
        new_index = new_index
    else:
        new_index = new_index + 1
        ind = row['Question_ID']
    data['Question_ID'][index] = new_index

C:\Users\USER\AppData\Local\Temp\ipykernel_12240\1323208810.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Question_ID'][index] = new_index


In [22]:
score5 = data[data['Score'] == 5.0]

new_data = pd.DataFrame(columns=['Question_ID', 'Question', 'Answer', 'Model_Answer', 'Score', 'Category'])

for index, row in score5.iterrows():
    # get the question id
    id = row['Question_ID']
    # get the original answer
    answer = row['Answer']

    # get the questions from the same id from data
    ques = data[data['Question_ID'] == id]

    # replace the model answer with answer
    ques['Model_Answer'] = answer

    # delete the row where the model answer is the same as the answer
    ques = ques[ques['Model_Answer'] != ques['Answer']]

    # add the new data to the new dataframe
    new_data = new_data.append(ques)

C:\Users\USER\AppData\Local\Temp\ipykernel_13196\3347836893.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques['Model_Answer'] = answer
C:\Users\USER\AppData\Local\Temp\ipykernel_13196\3347836893.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(ques)
C:\Users\USER\AppData\Local\Temp\ipykernel_13196\3347836893.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques['Model_Answer'] 

In [53]:
score5 = data[data['Score'] == 5.0]

new_data = pd.DataFrame(columns=['Question_ID', 'Question', 'Answer', 'Model_Answer', 'Score', 'Category', 'LengthRatio', 'Model_Answer_Embed', 'Answer_Embed'])

for index, row in score5.iterrows():
    # get the question id
    id = row['Question_ID']
    # get the original answer
    answer = row['Answer']
    model_ans_embed = row['Model_Answer_Embed']

    # get the questions from the same id from data
    ques = data[data['Question_ID'] == id]

    # replace the model answer with answer
    ques['Model_Answer'] = answer
    for index, row in ques.iterrows():
        ques['Model_Answer_Embed'][index] = model_ans_embed

    # delete the row where the model answer is the same as the answer
    ques = ques[ques['Model_Answer'] != ques['Answer']]

    # length ratio
    ques['LengthRatio'] = ques['Answer'].apply(lambda x: len(x) / len(row['Model_Answer']))

    # cosine similarity
    ques['Cosine_Similarity'] = ques.apply(lambda x: pairwise.cosine_similarity(x['Model_Answer_Embed'], x['Answer_Embed'])[0][0], axis=1)

    # add the new data to the new dataframe
    new_data = new_data.append(ques)

C:\Users\USER\AppData\Local\Temp\ipykernel_12240\3647376092.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques['Model_Answer'] = answer
C:\Users\USER\AppData\Local\Temp\ipykernel_12240\3647376092.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques['Model_Answer_Embed'][index] = model_ans_embed
C:\Users\USER\AppData\Local\Temp\ipykernel_12240\3647376092.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [57]:
# combine the new dataframe with the original dataframe
data = data.append(new_data)

C:\Users\USER\AppData\Local\Temp\ipykernel_12240\2175373774.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_data)


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36931 entries, 0 to 2734
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Question_ID         36931 non-null  object 
 1   Question            36931 non-null  object 
 2   Model_Answer        36931 non-null  object 
 3   Answer              36931 non-null  object 
 4   Score               36931 non-null  object 
 5   Category            36931 non-null  object 
 6   Model_Answer_Embed  36931 non-null  object 
 7   Answer_Embed        36931 non-null  object 
 8   Cosine_Similarity   36931 non-null  float32
 9   LengthRatio         34196 non-null  object 
dtypes: float32(1), object(9)
memory usage: 3.0+ MB


In [59]:
# create a new data frame without score, model_answer_embed, answer_embed, question, question_id
new_data = pd.DataFrame(columns=['Answer', 'Model_Answer', 'Category', 'LengthRatio', 'Cosine_Similarity'])

new_data = new_data.append(data[['Answer', 'Model_Answer', 'Category', 'LengthRatio', 'Cosine_Similarity']])


C:\Users\USER\AppData\Local\Temp\ipykernel_12240\215600557.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data[['Answer', 'Model_Answer', 'Category', 'LengthRatio', 'Cosine_Similarity']])


In [60]:
#remove duplicate rows
new_data = new_data.drop_duplicates()

In [61]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33686 entries, 0 to 2734
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Answer             33686 non-null  object
 1   Model_Answer       33686 non-null  object
 2   Category           33686 non-null  object
 3   LengthRatio        31036 non-null  object
 4   Cosine_Similarity  33686 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [62]:
new_data.to_csv('./improvedds_v3.csv', index=False)

In [3]:
data = pd.read_csv('../Data/improvedds_v3.csv', encoding="ISO-8859-1")

In [1]:
import pandas as pd

In [24]:
#load the file newds12 text file
data = pd.read_csv('../Data/newds12.txt', sep='\t')

In [25]:
#if relatedness_score >= 4 then category = 2
#if relatedness_score >=3 and relatedness_score < 4 then category = 1
#if relatedness_score < 3 then category = 0

data['Category'] = data['relatedness_score'].apply(lambda x: 2 if x >= 4 else (1 if x >= 3 and x < 4 else 0))
data['Model_Answer'] = data['sentence_A']
data['Answer'] = data['sentence_B']

In [26]:
#column names
data.columns

Index(['pair_ID', 'sentence_A', 'sentence_B', 'relatedness_score',
       'entailment_judgment', 'Category', 'Model_Answer', 'Answer'],
      dtype='object')

In [27]:
new_d = new_data

In [31]:
new_data

,Model_Answer,Answer,Category
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,2
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,1
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,2
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,1
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,1
...,...,...,...
496,Remove all the duplicate columns in the table ...,In first normalization remove all the duplicat...,1
497,Remove all the duplicate columns in the table ...,In first normalization remove all the duplicat...,0
498,Remove all the duplicate columns in the table ...,All the duplicated rows are removed in the fir...,0
499,Remove all the duplicate columns in the table ...,I don't have an idea about the question,0


In [28]:
#create new dataframe with only the columns we need
#Model_Answer, Answer, Category
new_data = pd.DataFrame(columns=['Model_Answer', 'Answer', 'Category'])

#Model_Answer is sentence_A
#Answer is sentence_B
new_data = new_data.append(data[['Model_Answer', 'Answer', 'Category']])

C:\Users\USER\AppData\Local\Temp\ipykernel_13196\162958309.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data[['Model_Answer', 'Answer', 'Category']])


In [29]:
new_data = new_data.append(new_d[['Model_Answer', 'Answer', 'Category']])

C:\Users\USER\AppData\Local\Temp\ipykernel_13196\3928169163.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(new_d[['Model_Answer', 'Answer', 'Category']])


In [32]:
#save the dataset as improveddsv6.csv
new_data.to_csv('../Data/improveddsv7.csv', index=False)


In [11]:
#load improveddsv5.csv
data2 = pd.read_csv('../Data/improveddsv5.csv', encoding="ISO-8859-1")

In [10]:
display(data.head())

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,Category
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL,2
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,1
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,2
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,1
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,1
